# Diamond Company Analytics Challenge

**Time:** 35 minutes

**Scenario:** You are the new data analyst at a prestigious diamond company. The CEO has called an urgent meeting and needs answers to critical business questions about the inventory.

**Your Mission:** Analyze the diamond inventory data and provide insights to guide strategic decisions.

In [1]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


## Section 1: Initial Inventory Assessment (5 minutes)

The CEO walks in: "Before we dive into specifics, I need to understand what we're working with. Give me the basics about our inventory database."

### Question 1: "How extensive is our inventory database?"

The CEO needs to know the scale of data you're analyzing.

In [4]:
# Your code here:

# Total number of records
total_records = len(df)

# Unique products
unique_products = df['Carat Weight'].n_unique() if 'Carat Weight' in df.columns else df.iloc[:, 0].n_unique()

# Unique categories
unique_categories = df['Cut'].n_unique() if 'Cut' in df.columns else 'Category column not found'

# Summary Output
print(f"Total Records: {total_records}")
print(f"Unique Products: {unique_products}")
print(f"Unique Categories: {unique_categories}")

Total Records: 6000
Unique Products: 196
Unique Categories: 5


### Question 2: "What information do we track about each diamond?"

List all the characteristics we record.

In [8]:
# Your code here:

# Display all column names
diamond_characteristics = df.columns

print("We track the following characteristics for each diamond:")
for col in diamond_characteristics:
    print(f"• {col}")

We track the following characteristics for each diamond:
• Carat Weight
• Cut
• Color
• Clarity
• Polish
• Symmetry
• Report
• Price


### Question 3: "When I look at one line in this database, what am I looking at?"

Explain in business terms:

**Your answer:** Each row represents: one single diamond and its details.

## Section 2: Strategic Business Questions (20 minutes)

The CEO continues: "Now let's get to the important questions. Our board meeting is tomorrow and I need concrete numbers."

### Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight."

Think about what measurement would help answer this.

In [16]:
# Your code here:

if 'Carat Weight' in df.columns:
    # Sum up total carats in inventory
    total_carats = df['Carat Weight'].sum()

    print(f" Total Carats in Inventory: {total_carats:,.2f} carats")

 Total Carats in Inventory: 8,007.12 carats


### Question 5: "Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?"

The CEO wants to know about variety, not quantity.

In [21]:
# Your code here:

# Check if 'Cut' column exists
if 'Cut' in df.columns:
    # Get unique cuts
    unique_cuts = list(df['Cut'].unique())
    num_unique_cuts = len(unique_cuts)

    print(f"We currently offer {num_unique_cuts} distinct diamond cuts.")
    print("Here is the list of cuts:")
    for cut in sorted(unique_cuts):
        print(f"• {cut}")
else:
    print(" Column 'cut' not found in the dataset.")

We currently offer 5 distinct diamond cuts.
Here is the list of cuts:
• Fair
• Good
• Ideal
• Signature-Ideal
• Very Good


### Question 6: "I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?"

In [26]:
# Your code here:

# Ensure 'Price' column exists
if 'Price' in df.columns:
    # Sort the DataFrame by price
    sorted_by_price = df.sort('Price')

    # Least valuable diamond
    least_valuable = sorted_by_price.row(0, named=True)

    # Most valuable diamond
    most_valuable = sorted_by_price.row(-1, named=True)

    print("Least Valuable Diamond:")
    print(least_valuable)

    print("\nMost Valuable Diamond:")
    print(most_valuable)

Least Valuable Diamond:
{'Carat Weight': 0.77, 'Cut': 'Good', 'Color': 'I', 'Clarity': 'VS1', 'Polish': 'VG', 'Symmetry': 'G', 'Report': 'AGSL', 'Price': 2184}

Most Valuable Diamond:
{'Carat Weight': 2.79, 'Cut': 'Ideal', 'Color': 'D', 'Clarity': 'IF', 'Polish': 'EX', 'Symmetry': 'EX', 'Report': 'GIA', 'Price': 101561}


### Question 7: "Which color grades should we focus on in our premium line? I need typical prices across our color spectrum."

The CEO wants to understand pricing patterns.

In [31]:
# Your code here:

# Check that necessary columns exist
if 'Color' in df.columns and 'Price' in df.columns:
    # Group by Color and calculate count, mean, and median price
    price_by_color = df.group_by('Color').agg([
        pl.len().alias("Count"),
        pl.col('Price').mean().alias("Mean Price"),
        pl.col('Price').median().alias("Median Price")
    ]).sort('Mean Price', descending=True)

    # Display the result
    print("Typical Prices by Color Grade:")
    display(price_by_color)
else:
    print("Required columns 'Color' and/or 'Price' not found in the dataset.")

Typical Prices by Color Grade:


Color,Count,Mean Price,Median Price
str,u32,f64,f64
"""D""",661,15255.783661,7567.0
"""F""",1013,12712.241856,8860.0
"""G""",1501,12520.050633,8800.0
"""E""",778,11539.190231,7059.5
"""H""",1079,10487.347544,7169.0
"""I""",968,8989.636364,6655.0


**Reflection:** After analyzing by color, how has your view of the data changed? What does each row in your result represent now?

**Your answer:** New grain = the diamond color grade.

## Section 3: Advanced Strategic Analysis (10 minutes)

The CEO leans forward: "These next questions will shape our strategy for next year."

### Question 8: "We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

In [35]:
# Your code here:

# Ensure necessary columns exist
required_cols = {'Clarity', 'Price', 'Carat Weight'}
if required_cols.issubset(df.columns):
    # Group by clarity and aggregate
    clarity_breakdown = df.group_by('Clarity').agg([
        pl.len().alias("Num Diamonds"),
        pl.col('Price').mean().alias("Avg Price"),
        pl.col('Price').median().alias("Median Price"),
        pl.col('Carat Weight').sum().alias("Total Carats")
    ]).sort('Avg Price', descending=False)

    # Display the result
    print("🔍 Clarity Grade Breakdown:")
    display(clarity_breakdown)

🔍 Clarity Grade Breakdown:


Clarity,Num Diamonds,Avg Price,Median Price,Total Carats
str,u32,f64,f64,f64
"""SI1""",2059,8018.864012,5417.0,2563.69
"""VS2""",1575,11809.053333,7568.0,2170.81
"""VS1""",1192,13694.113255,9245.0,1682.74
"""VVS2""",666,14142.177177,10266.0,876.58
"""VVS1""",285,16845.680702,11142.0,389.43
"""IF""",219,22105.844749,12647.0,316.0
"""FL""",4,63776.0,62371.5,7.87


### Question 9: "My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive."

In [39]:
# Your code here:

# Ensure required columns are present
required_cols = {'Cut', 'Color', 'Price'}
if required_cols.issubset(df.columns):
    # Group by cut and color, then calculate price stats using Polars
    price_by_cut_color = df.group_by(['Cut', 'Color']).agg([
        pl.len().alias("count"),
        pl.col('Price').mean().alias("avg_price"),
        pl.col('Price').median().alias("median_price"),
        pl.col('Price').min().alias("min_price"),
        pl.col('Price').max().alias("max_price")
    ]).sort(by='avg_price', descending=False)

    # Display the result
    print("💎 Pricing Structure by Cut-Color Combination:")
    display(price_by_cut_color)

💎 Pricing Structure by Cut-Color Combination:


Cut,Color,count,avg_price,median_price,min_price,max_price
str,str,u32,f64,f64,i64,i64
"""Fair""","""I""",16,4573.1875,3801.0,3166,8999
"""Fair""","""E""",32,5370.625,4908.5,3743,12829
"""Fair""","""H""",24,5908.5,4358.0,2594,17242
"""Fair""","""D""",12,6058.25,5040.0,4105,19079
"""Fair""","""F""",24,6063.625,4628.0,2648,29368
…,…,…,…,…,…,…
"""Signature-Ideal""","""F""",38,13247.947368,10433.0,3108,59538
"""Ideal""","""G""",690,13570.310145,9510.5,2870,46861
"""Ideal""","""F""",363,14729.426997,10360.0,2709,60188


**Reflection:** How specific is your analysis now? What level of detail does each row represent?

**Your answer:** New grain = cut-color combination.

### Question 10: URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."

This is critical for tomorrow's negotiation.

In [44]:
# Your code here:

# Ensure required columns are present
required_cols = {'Cut', 'Color', 'Price'}
if required_cols.issubset(df.columns):
    # Group by cut and color to summarize
    segment_summary = df.group_by(['Cut', 'Color']).agg([
        pl.sum('Price').alias('total_inventory_value'),
        pl.len().alias('diamond_count'),
    ]).sort(by='total_inventory_value', descending=True)

    # Get top 5
    top_5_segments = segment_summary.head(5)

    print("Top 5 Most Valuable Inventory Segments (by Cut-Color):")
    display(top_5_segments)

Top 5 Most Valuable Inventory Segments (by Cut-Color):


Cut,Color,total_inventory_value,diamond_count
str,str,i64,u32
"""Ideal""","""G""",9363514,690
"""Very Good""","""G""",7140620,578
"""Very Good""","""F""",5648327,455
"""Ideal""","""F""",5346782,363
"""Ideal""","""H""",5279687,458


## Debrief

Congratulations! You've just completed your first day as the diamond company's data analyst.

Remember:
- CEOs speak in business language, not technical terms
- Always understand what each row represents before analyzing
- Business questions contain clues about which operations to use